<h1>Preparation of the German Twitter Data</h1>

<p>This file is for preparing the german tweets. The preparation is mainly for training my own model for sentiment detection with Flair. </p>

In [1]:
import csv
import numpy as np
import pandas as pd
import re
import nltk

<h1>Einlesen und bestimmung der Felder</h1>
<p>Durch die weitern Operatoren, kann ich die einzelnen Felder in DF bestimmen. Durch die Lambda Funktion werden die Str zu Listen, was die spätere verarbeitung erleichtert. 
    Durch die Parse_dates Funktion wird das Datumsfeld zu einem Timestamp, was die weiter Verarbeitung wesentlich leichter macht.</p>

In [2]:
df_manager_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_DE/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])

In [3]:
df_manager_de.head()

conversation_id   author_id                created_at  \
0  703311366262800384  1392131804 2016-02-26 20:11:11+00:00   
1  702405572012974080  1053723204 2016-02-24 08:11:53+00:00   
2  701784983829143553   554491983 2016-02-22 15:05:53+00:00   
3  701446667170615296  3365669884 2016-02-21 16:41:32+00:00   
4  700597812556386304  4858383167 2016-02-19 08:28:29+00:00   

             tweet_id lang  retweet_count  reply_count  like_count  \
0  703311366262800384   de              0            0           2   
1  702405572012974080   de              0            0           0   
2  701784983829143553   de              0            0           0   
3  701446667170615296   de              0            0           1   
4  700597812556386304   de              2            0           0   

   quote_count              source   tweet_type   referenced_tweet_id  \
0            0  Twitter Web Client       [None]                [None]   
1            0  Twitter Web Client       [None]                [None]   
2            0            Facebook       [None]                [None]   
3            0  Twitter Web Client       [None]                [None]   
4            0  Twitter for iPhone  [retweeted]  [700578193334411264]   

                                                text  
0  #Wirecard-Chef Markus Braun pumpt irrwitzige S...  
1  Jetzt günstig einsammeln und später absahnen? ...  
2  Morgen startet die neue Reihe R²-JazzCube! Vol...  
3  die Y-Achse des  #prusai3 nimmt Formen an :-) ...  
4  RT @aktionaer: #Wirecard: Was führt der Vorsta...

<br>
<p>Prüfen welche Datenfelder in den einzelnen Zeilen enthalten sind. War wichtig um diese anpassen zu können.</p>

In [4]:
print("conversation_id: ", df_manager_de.iloc[0]["conversation_id"], "type: ", type(df_manager_de.iloc[0]["conversation_id"]))
print("author_id: ", df_manager_de.iloc[0]["author_id"], "type: ", type(df_manager_de.iloc[0]["author_id"]))
print("created_at: ", df_manager_de.iloc[0]["created_at"], "type: ", type(df_manager_de.iloc[0]["created_at"]))
print("tweet_id: ", df_manager_de.iloc[0]["tweet_id"], "type: ", type(df_manager_de.iloc[0]["tweet_id"]))
print("lang: ", df_manager_de.iloc[0]["lang"], "type: ", type(df_manager_de.iloc[0]["lang"]))
print("retweet_count: ", df_manager_de.iloc[0]["retweet_count"], "type: ", type(df_manager_de.iloc[0]["retweet_count"]))
print("reply_count: ", df_manager_de.iloc[0]["reply_count"], "type: ", type(df_manager_de.iloc[0]["reply_count"]))
print("like_count: ", df_manager_de.iloc[0]["like_count"], "type: ", type(df_manager_de.iloc[0]["like_count"]))
print("quote_count: ", df_manager_de.iloc[0]["quote_count"], "type: ", type(df_manager_de.iloc[0]["quote_count"]))
print("source: ", df_manager_de.iloc[0]["source"], "type: ", type(df_manager_de.iloc[0]["source"]))
print("tweet_type: ", df_manager_de.iloc[0]["tweet_type"], "type: ", type(df_manager_de.iloc[0]["tweet_type"]))
print("referenced_tweet_id: ", df_manager_de.iloc[0]["referenced_tweet_id"], "type: ", type(df_manager_de.iloc[0]["referenced_tweet_id"]))
print("text: ", df_manager_de.iloc[0]["text"], "type: ", type(df_manager_de.iloc[0]["text"]))

conversation_id:  703311366262800384 type:  <class 'numpy.int64'>
author_id:  1392131804 type:  <class 'numpy.int64'>
created_at:  2016-02-26 20:11:11+00:00 type:  <class 'pandas._libs.tslibs.timestamps.Timestamp'>
tweet_id:  703311366262800384 type:  <class 'numpy.int64'>
lang:  de type:  <class 'str'>
retweet_count:  0 type:  <class 'numpy.int64'>
reply_count:  0 type:  <class 'numpy.int64'>
like_count:  2 type:  <class 'numpy.int64'>
quote_count:  0 type:  <class 'numpy.int64'>
source:  Twitter Web Client type:  <class 'str'>
tweet_type:  ['None'] type:  <class 'list'>
referenced_tweet_id:  ['None'] type:  <class 'list'>
text:  #Wirecard-Chef Markus Braun pumpt irrwitzige Summen in sein Unternehmen. Woher stammt das Geld? @SchreiberDohms  https://t.co/VdsV4nvBe4 type:  <class 'str'>


<br>
<h3>Nicht Kontextbezogene Tweets löschen</h3>
<p>So kann ich prüfen ob bestimmte Begriffe in den Texten enthalten sind und mir die conversation_id ausgeben lassen. Diese lasse ich dann löschen und auch zugehörige Tweets, da diese nicht in den Kontext passen.</p>
<p>Zu löschende Begriffe</p>
<ul>
    <li>BVB</li>
    <li>Fußball</li>
    <li>Fussball</li>
    <li>Dortmund</li>
</ul>

In [5]:
for each in range(len(df_manager_de)):
    if ("BVB" or "Fußball" or "Fussball" or "Dortmund") in df_manager_de.iloc[each]["text"]:
        print(df_manager_de.iloc[each]["conversation_id"])

722861750723284993
722858374807822336
739490204180992000
798533984854544384
798522085756678144
798516440277995520
926889966369689600
984792434533191680
984778759629885440
1088115106519633920
1260905841433198592
1260900850077179905
1260881271372288000
1260880569224900610
1278590734430547969
1278581125951225858
1278578905042034688
1278569106493640704
1278564053112295425
1278443988769542151
1278436721462513666
1278435775441494016
1278435182056542212
1278434921594454019
1278434819685441537
1278434701418651650


In [6]:
ids_without_context_list = []
for each in range(len(df_manager_de)):
    if ("BVB" or "Fußball" or "Fussball" or "Dortmund") in df_manager_de.iloc[each]["text"]:
        ids_without_context_list.append(each)
        #print(df_manager_de.iloc[each]["conversation_id"])

In [15]:
print(ids_without_context_list)

[14, 15, 21, 25, 26, 27, 52, 84, 85, 362, 3675, 3676, 3677, 3678, 14194, 14199, 14200, 14202, 14203, 14207, 14208, 14209, 14210, 14211, 14212, 14213]


In [14]:
df_manager_de.iloc[ids_without_context_list]["referenced_tweet_id"]

14        [722858374807822336]
15                      [None]
21        [739490204180992000]
25        [798522085756678144]
26                      [None]
27                      [None]
52        [926889717286752256]
84                      [None]
85        [984779935964377091]
362                     [None]
3675     [1260880569224900610]
3676     [1260880569224900610]
3677     [1260880569224900610]
3678                    [None]
14194    [1278434701418651650]
14199    [1278434701418651650]
14200    [1278434701418651650]
14202    [1278434701418651650]
14203    [1278434701418651650]
14207    [1278434701418651650]
14208    [1278434701418651650]
14209    [1278434701418651650]
14210    [1278434701418651650]
14211    [1278434701418651650]
14212    [1278434701418651650]
14213                   [None]
Name: referenced_tweet_id, dtype: object

<br>
<h4>Check</h4>
<p>Aufrufen einer Line um zu prüfen, welche Informationen enthalten sind.<br>
    Den Index abrufen um mir nur den Text in voller Länge anzeigen zu lassen, da dieser sonst gekürzt ist.</p>
   

In [35]:
df_manager_de.loc[df_manager_de["conversation_id"] == 1278435775441494016]

conversation_id            author_id                created_at  \
14209  1278435775441494016  1149720065274732546 2020-07-01 21:10:15+00:00   

                  tweet_id lang  retweet_count  reply_count  like_count  \
14209  1278435775441494016   de             11            0           0   

       quote_count              source   tweet_type    referenced_tweet_id  \
14209            0  Twitter for iPhone  [retweeted]  [1278434701418651650]   

                                                    text  
14209  RT @RNBVB: 👨🏻‍⚕️ Dr. Markus #Braun ist vom Nac...

In [49]:
df_manager_de.loc[df_manager_de.index[df_manager_de["conversation_id"] == 1278435775441494016].tolist()[0]]["text"]

'RT @RNBVB: 👨🏻\u200d⚕️ Dr. Markus #Braun ist vom Nachrichtenmagazin #Focus als "Top-Mediziner" in Deutschland ausgezeichnet worden. Zuletzt arbei…'

In [48]:
print(df_manager_de.index[df_manager_de["conversation_id"] == 1278435775441494016].tolist()[0])

14209


In [36]:
df_manager_de.loc[14209]["text"]

'RT @RNBVB: 👨🏻\u200d⚕️ Dr. Markus #Braun ist vom Nachrichtenmagazin #Focus als "Top-Mediziner" in Deutschland ausgezeichnet worden. Zuletzt arbei…'

<br>
<h3>Löschen von Retweets</h3>
<p>Alle Retweets löschen, da diese in diesem Schritt nicht benötigt werden.</p>

In [53]:
df_manager_de.iloc[4]

conversation_id                                       700597812556386304
author_id                                                     4858383167
created_at                                     2016-02-19 08:28:29+00:00
tweet_id                                              700597812556386304
lang                                                                  de
retweet_count                                                          2
reply_count                                                            0
like_count                                                             0
quote_count                                                            0
source                                                Twitter for iPhone
tweet_type                                                   [retweeted]
referenced_tweet_id                                 [700578193334411264]
text                   RT @aktionaer: #Wirecard: Was führt der Vorsta...
Name: 4, dtype: object

In [83]:
counter = 0
retweet_list = []
for each in range(len(df_manager_de)):
    if "retweeted" in df_manager_de.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)

5455


In [79]:
print(type(df_manager_de.iloc[0]["tweet_type"]))

<class 'list'>


In [85]:
df_manager_de.iloc[retweet_list]

conversation_id            author_id                created_at  \
4       700597812556386304           4858383167 2016-02-19 08:28:29+00:00   
5       700595335081697280           1053723204 2016-02-19 08:18:38+00:00   
9       695225816867799040            215669569 2016-02-04 12:42:06+00:00   
14      722861750723284993            859092655 2016-04-20 18:57:26+00:00   
19      732214603153805312           2844082392 2016-05-16 14:22:19+00:00   
...                    ...                  ...                       ...   
14293  1278247164666417153           2530028126 2020-07-01 08:40:47+00:00   
14295  1278235640036036608   701049437943177216 2020-07-01 07:54:59+00:00   
14296  1278230970777440256             52016801 2020-07-01 07:36:26+00:00   
14299  1278224614888558592  1140518463259258880 2020-07-01 07:11:11+00:00   
14301  1278209796953575424            267046768 2020-07-01 06:12:18+00:00   

                  tweet_id lang  retweet_count  reply_count  like_count  \
4       700597812556386304   de              2            0           0   
5       700595335081697280   de              2            0           0   
9       695225816867799040   de              1            0           0   
14      722861750723284993   de              1            0           0   
19      732214603153805312   de              1            0           0   
...                    ...  ...            ...          ...         ...   
14293  1278247164666417153   de              1            0           0   
14295  1278235640036036608   de              9            0           0   
14296  1278230970777440256   de              4            0           0   
14299  1278224614888558592   de              4            0           0   
14301  1278209796953575424   de             25            0           0   

       quote_count               source   tweet_type    referenced_tweet_id  \
4                0   Twitter for iPhone  [retweeted]   [700578193334411264]   
5                0   Twitter for iPhone  [retweeted]   [700578193334411264]   
9                0     Tweetbot for Mac  [retweeted]   [694992399874056192]   
14               0  Twitter for Android  [retweeted]   [722858374807822336]   
19               0            RoundTeam  [retweeted]   [732209352036192260]   
...            ...                  ...          ...                    ...   
14293            0   Twitter for iPhone  [retweeted]  [1278247102091534336]   
14295            0  Twitter for Android  [retweeted]  [1276379791558610944]   
14296            0  Twitter for Android  [retweeted]  [1278089329517133825]   
14299            0      Twitter Web App  [retweeted]  [1278089329517133825]   
14301            0      Twitter Web App  [retweeted]  [1275363843632496641]   

                                                    text  
4      RT @aktionaer: #Wirecard: Was führt der Vorsta...  
5      RT @aktionaer: #Wirecard: Was führt der Vorsta...  
9      RT @MarkusB1980: Der Rahmen vom #prusai3 ist z...  
14     RT @RylaH90: "Keiner tackert so schnell wie Ma...  
19     RT @MarkusB1980: "kleiner" Bericht zum Grillen...  
...                                                  ...  
14293  RT @AlexanderPoel: Seit heute Morgen durchsuch...  
14295  RT @ReneBenderRB: Wer nur einen Artikel über d...  
14296  RT @aegidius: Wer erinnert sich denn noch dara...  
14299  RT @aegidius: Wer erinnert sich denn noch dara...  
14301  RT @MarioLindner82: Der Kurz-Berater und ehema...  

[5455 rows x 13 columns]

In [86]:
df_manager_de.drop(retweet_list)

conversation_id            author_id                created_at  \
0       703311366262800384           1392131804 2016-02-26 20:11:11+00:00   
1       702405572012974080           1053723204 2016-02-24 08:11:53+00:00   
2       701784983829143553            554491983 2016-02-22 15:05:53+00:00   
3       701446667170615296           3365669884 2016-02-21 16:41:32+00:00   
6       700578193334411264             19811837 2016-02-19 07:10:32+00:00   
...                    ...                  ...                       ...   
14303  1278189179327954944            212758794 2020-07-01 04:50:22+00:00   
14304  1278182059572432896             58417079 2020-07-01 04:22:05+00:00   
14305  1278164531613782017            386165847 2020-07-01 03:12:26+00:00   
14306  1278158825661566979  1245423304304926727 2020-07-01 02:49:45+00:00   
14307  1278150802956324864            163466981 2020-07-01 02:17:52+00:00   

                  tweet_id lang  retweet_count  reply_count  like_count  \
0       703311366262800384   de              0            0           2   
1       702405572012974080   de              0            0           0   
2       701784983829143553   de              0            0           0   
3       701446667170615296   de              0            0           1   
6       700578193334411264   de              2            0           0   
...                    ...  ...            ...          ...         ...   
14303  1278189179327954944   de              0            0           0   
14304  1278182059572432896   de              0            0           1   
14305  1278164531613782017   de              0            0           0   
14306  1278158825661566979   de              0            0           0   
14307  1278150802956324864   de              0            0           0   

       quote_count              source tweet_type referenced_tweet_id  \
0                0  Twitter Web Client     [None]              [None]   
1                0  Twitter Web Client     [None]              [None]   
2                0            Facebook     [None]              [None]   
3                0  Twitter Web Client     [None]              [None]   
6                0  Twitter Web Client     [None]              [None]   
...            ...                 ...        ...                 ...   
14303            0     Twitter Web App     [None]              [None]   
14304            0            LinkedIn     [None]              [None]   
14305            0               IFTTT     [None]              [None]   
14306            0     Blog2Social APP     [None]              [None]   
14307            0              Aktien     [None]              [None]   

                                                    text  
0      #Wirecard-Chef Markus Braun pumpt irrwitzige S...  
1      Jetzt günstig einsammeln und später absahnen? ...  
2      Morgen startet die neue Reihe R²-JazzCube! Vol...  
3      die Y-Achse des  #prusai3 nimmt Formen an :-) ...  
6      #Wirecard: Was führt der Vorstand im Schilde? ...  
...                                                  ...  
14303  #Wirecard wird womöglich gestückelt an Investo...  
14304  Das Ende eines großen Traums zeichnet sich ab....  
14305  Wirecard kündigt Markus Braun, Insolvenzverwal...  
14306  Kündigung für Markus Braun: Der Aufsichtsrat v...  
14307  DGAP-Adhoc: Wirecard AG: Kündigung Anstellungs...  

[8853 rows x 13 columns]

In [91]:
quoted_list = []
for each in range(len(df_manager_de)):
    if "quoted" in df_manager_de.iloc[each]["tweet_type"]:
        quoted_list.append(each)

In [92]:
df_manager_de.iloc[quoted_list]

conversation_id            author_id                created_at  \
52      926889966369689600           2482271094 2017-11-04 19:12:43+00:00   
68      961528788407537664           1053723204 2018-02-08 09:15:02+00:00   
364    1087241825981743104  1027893295538679811 2019-01-21 06:53:45+00:00   
366    1086563243303288833           3418888874 2019-01-19 09:57:19+00:00   
387    1098886062590623744            399025508 2019-02-22 10:03:48+00:00   
...                    ...                  ...                       ...   
14086  1278988351257948161           3019978209 2020-07-03 09:46:00+00:00   
14108  1278917293423165440           3019978209 2020-07-03 05:03:38+00:00   
14204  1278561534558515201           3019978209 2020-07-02 05:29:59+00:00   
14254  1278304420535644160  1277511900482461696 2020-07-01 12:28:18+00:00   
14300  1278223650328584196   844906242900418563 2020-07-01 07:07:21+00:00   

                  tweet_id lang  retweet_count  reply_count  like_count  \
52      926889966369689600   de              0            0           5   
68      961528788407537664   de              2            0           1   
364    1087241825981743104   de              1            2           5   
366    1086563243303288833   de              0            0           1   
387    1098886062590623744   de              0            1           0   
...                    ...  ...            ...          ...         ...   
14086  1278988351257948161   de              0            1           8   
14108  1278917293423165440   de              0            2          12   
14204  1278561534558515201   de              0            0           7   
14254  1278304420535644160   de              0            0           0   
14300  1278223650328584196   de              1            0           1   

       quote_count              source tweet_type    referenced_tweet_id  \
52               0  Twitter Web Client   [quoted]   [926889717286752256]   
68               0  Twitter for iPhone   [quoted]   [961521761325076480]   
364              0  Twitter for iPhone   [quoted]  [1087216963800256513]   
366              0  Twitter Web Client   [quoted]  [1086295585529856000]   
387              0           TweetDeck   [quoted]  [1098839333518430208]   
...            ...                 ...        ...                    ...   
14086            0     Twitter Web App   [quoted]  [1278757395892703235]   
14108            1     Twitter Web App   [quoted]  [1278785083391041565]   
14204            0     Twitter Web App   [quoted]  [1278557061857587206]   
14254            0  Twitter for iPhone   [quoted]  [1276561563818831872]   
14300            0     Twitter Web App   [quoted]  [1278198537747140608]   

                                                    text  
52     Wusste gar nicht, dass Dr. Markus Braun auch F...  
68     CEO Markus Braun kauft wieder im 7-stelligen V...  
364    Kabelentstörung im Katastrophengebiet- @Jodl s...  
366    Auch von mir ein dickes Lob an die Exoten unte...  
387    Lesenswertes Interview von @fruehaufmarkus mit...  
...                                                  ...  
14086  "Es heißt, er habe nicht gerne mit Karten geza...  
14108  Man stelle sich vor, Markus Braun hätte die Ba...  
14204  Ich hatte einen Beratervertrag mit Jan Marsale...  
14254  War zwar Scheißprodukt, aber @_MarkusBraun wol...  
14300  Stimmt - siehe auch: https://t.co/fdSrhIJXHq -...  

[351 rows x 13 columns]

In [23]:
source_unique_list = list(np.unique(source_list))

In [27]:
df_user_manager_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_DE/user.csv", sep=";")

In [28]:
df_user_manager_de.head()

user_id        username                  name  follower  following  \
0  1392131804  capitalMagazin               Capital      7669        436   
1  1053723204         l_m_s15           Lukas Spang      2292        237   
2   554491983      bvb_remmel            bvb.remmel        70         66   
3  3365669884     MarkusB1980          Markus Braun        82        181   
4  4858383167      QuasselDax  PrivateTraderContact         8         17   

   tweet_cound  list_count  verified                 created_at  
0         6287         212     False  2013-04-30 13:55:36+00:00  
1        39499          92     False  2013-01-01 23:31:19+00:00  
2         1831           0     False  2012-04-15 15:05:15+00:00  
3          620          13     False  2015-07-08 08:43:24+00:00  
4           61           4     False  2016-01-28 20:15:04+00:00

In [35]:
df_user_manager_de["name"].where(df_user_manager_de["name"] == "Markus Braun")

0                 NaN
1                 NaN
2                 NaN
3        Markus Braun
4                 NaN
             ...     
10407             NaN
10408             NaN
10409             NaN
10410             NaN
10411             NaN
Name: name, Length: 10412, dtype: object

In [54]:
df_user_manager_de[df_user_manager_de["name"].values == "Markus"]

user_id  username    name  follower  following  tweet_cound  \
5980  348086065  Leineems  Markus        19        156          248   

      list_count  verified                 created_at  
5980           1     False  2011-08-03 21:16:41+00:00

In [48]:
df_user_manager_de.iloc[18]

user_id                       3365669884
username                     MarkusB1980
name                        Markus Braun
follower                              82
following                            181
tweet_cound                          620
list_count                            13
verified                           False
created_at     2015-07-08 08:43:24+00:00
Name: 18, dtype: object